- Flip coin
- Dice Roll

In [1]:
# Data manipulation libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data import library
import yfinance as yf

#Data visualisation
import plotly.graph_objs as go

from binance.client import Client
from binance.enums import HistoricalKlinesType

In [2]:
api_key = 'y9m94JvyDsSRyHZUQjNN1lVfDnduNMFyQxz83EW650XKwgbIXxB85xGv7CDQfW8Z'
api_secret = 'cHvzTpnE3Qp468ZcAIadtIQSKvV6HdbXsnwbTAhlTCg9Q0uydcxVKUwP7rqLrcpl'

klines_column_names = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 
                       'Number of trades', 'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore']

client = Client(api_key, api_secret, {"timeout": 60})

In [143]:
start_date = '1 March, 2022'
end_date = '5 March, 2022'

In [144]:
klines = client.get_historical_klines(symbol = "ETHUSDT", 
                             interval = Client.KLINE_INTERVAL_15MINUTE, 
                             start_str = start_date,
                             end_str = end_date,
                             klines_type = HistoricalKlinesType.FUTURES)

In [145]:
df_klines = pd.DataFrame(klines, columns=klines_column_names).set_index('Open time')
df_klines.index = pd.to_datetime(df_klines.index, unit='ms')
df_klines = df_klines.astype({'Open':'double', 'High':'double', 'Low':'double', 'Close':'double', 'Volume': 'double'})

df = df_klines[['Open', 'High', 'Low', 'Close', 'Volume']]

In [146]:
def dice_trade(data, threshold=4):
    data = data.assign(random=np.random.randint(1,7,len(data)))
    data['profit'] = 0

    data.loc[data.random < threshold, 'profit'] = (data[data.random < threshold].Close - data[data.random < threshold].Open) / data[data.random < threshold].Open + 1
    data.loc[data.random >= threshold, 'profit'] = (data[data.random >= threshold].Open - data[data.random >= threshold].Close) / data[data.random >= threshold].Open + 1
    return data

In [147]:
def coinflip_trade(data):
    data = data.assign(random=np.random.randint(1,3,len(data)))
    data['profit'] = 0

    data.loc[data.random == 1, 'profit'] = (data[data.random == 1].Close - data[data.random == 1].Open) / data[data.random == 1].Open + 1
    data.loc[data.random == 2, 'profit'] = (data[data.random == 2].Open - data[data.random == 2].Close) / data[data.random == 2].Open + 1
    return data

In [159]:
dice_trade(df, threshold=5).profit.cumprod()

Open time
2022-03-01 00:00:00    1.012876
2022-03-01 00:15:00    1.009243
2022-03-01 00:30:00    1.008267
2022-03-01 00:45:00    1.009101
2022-03-01 01:00:00    1.016763
                         ...   
2022-03-04 23:00:00    1.105321
2022-03-04 23:15:00    1.106791
2022-03-04 23:30:00    1.103657
2022-03-04 23:45:00    1.103177
2022-03-05 00:00:00    1.100256
Name: profit, Length: 385, dtype: float64

In [149]:
coinflip_trade(df).profit.cumprod()

Open time
2022-03-01 00:00:00    0.987124
2022-03-01 00:15:00    0.990665
2022-03-01 00:30:00    0.989707
2022-03-01 00:45:00    0.990525
2022-03-01 01:00:00    0.998046
                         ...   
2022-03-04 23:00:00    0.892057
2022-03-04 23:15:00    0.890871
2022-03-04 23:30:00    0.893393
2022-03-04 23:45:00    0.893004
2022-03-05 00:00:00    0.890639
Name: profit, Length: 385, dtype: float64